<a href="https://colab.research.google.com/github/garmartirosy/GithubActions/blob/main/Factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymrio

In [ ]:
import pymrio
import pandas as pd
import uuid

pymrio.download_exiobase3(storage_folder='content', system='pxp', years=[2012])



exio_data = pymrio.parse_exiobase3(path="/content/content/IOT_2012_pxp.zip")

exio_data.satellite.F # Factors of productions/stressors/impacts
df = exio_data.satellite.F

df.columns = ['_'.join(col).strip() for col in df.columns.values]
df.reset_index(inplace=True)

melted_df = df.melt(
    id_vars=['stressor'],  # Keep the 'stressor' column fixed
    var_name='Region_Sector',  # New column for region-sector combinations
    value_name='Value'  # Column for numerical values
)


melted_df[['Region', 'Sector']] = melted_df['Region_Sector'].str.extract(r'([A-Z]+)_(.*)')


melted_df = melted_df.drop(columns=['Region_Sector'])

melted_df = melted_df.rename(columns={'stressor': 'Stressor'})

# Reorder columns for clarity
melted_df = melted_df[['Stressor', 'Region', 'Sector', 'Value']]

region_mapping_path = '/content/exio_country_names.csv'  # Replace with actual path
region_mapping = pd.read_csv(region_mapping_path)



# Merge region mapping with the reshaped DataFrame
melted_df = melted_df.merge(region_mapping, how='left', left_on='Region', right_on='CountryCode')

# Replace the 'Region' column with full country names
melted_df = melted_df.drop(columns=['Region', 'CountryCode']).rename(columns={'Country': 'Region'})


melted_df['Value'] = pd.to_numeric(melted_df['Value'], errors='coerce')

# Handle missing values (replace NaN with 0)
melted_df = melted_df.fillna(0)



# Example reshaped data with additional fields (use your loaded DataFrame `melted_df`)
# For demonstration, let's assume `melted_df` contains Region, Sector, Stressor, Value
melted_df['Year'] = 2019  # Add year (modify as needed)
melted_df['Unit'] = 'kg'  # Add unit (modify as needed)
melted_df['ReferenceCurrency'] = 'USD'  # Add currency
melted_df['PriceType'] = 'Basic'  # Add price type
melted_df['Context'] = 'emission/air'  # Add context

# Generate FlowUUIDs for each unique flowable (Stressor)
melted_df['FlowUUID'] = [str(uuid.uuid4()) for _ in range(len(melted_df))]

# Rename Stressor to Flowable for consistency
melted_df = melted_df.rename(columns={'Stressor': 'Flowable', 'Value': 'FlowAmount'})

# Select and reorder columns
factors_table = melted_df[[
    'Region', 'Sector', 'Year', 'Unit', 'ReferenceCurrency', 'PriceType',
    'Flowable', 'Context', 'FlowUUID', 'FlowAmount'
]]

In [ ]:
factors_table[factors_table['FlowAmount'] != 0].head(200)

,Region,Sector,Year,Unit,ReferenceCurrency,PriceType,Flowable,Context,FlowUUID,FlowAmount
69,Austria,Paddy rice,2019,kg,USD,Basic,CH4 - non combustion - Mining of antracite - air,emission/air,d1fcd6d6-4afb-4961-94af-93f593c92bcf,1.294620e-06
70,Austria,Paddy rice,2019,kg,USD,Basic,CH4 - non combustion - Mining of bituminous co...,emission/air,75babab1-e707-4d87-8607-9cad072b0394,1.248724e-07
71,Austria,Paddy rice,2019,kg,USD,Basic,CH4 - non combustion - Mining of coking coal -...,emission/air,da3f143e-c8c4-4983-8b5d-9d3747f26d03,1.507088e-07
72,Austria,Paddy rice,2019,kg,USD,Basic,CH4 - non combustion - Mining of lignite (brow...,emission/air,15f587b4-7452-42f7-b32e-5daa587f8b35,3.836979e-07
73,Austria,Paddy rice,2019,kg,USD,Basic,CH4 - non combustion - Mining of sub-bituminou...,emission/air,c5a66017-c062-4ffe-91d6-77b6d2401a26,2.653970e-07
1113,Austria,Wheat,2019,kg,USD,Basic,Taxes less subsidies on products purchased: Total,emission/air,c349a4f2-b17c-4013-814c-b7cbd42197a0,5.948006e+00
1114,Austria,Wheat,2019,kg,USD,Basic,Other net taxes on production,emission/air,e2b77fed-08f4-4bf9-97e8-8504f970868e,-1.213726e+01
1115,Austria,Wheat,2019,kg,USD,Basic,"Compensation of employees; wages, salaries, & ...",emission/air,6315d198-c07f-492a-a3c7-d433cca2fb62,1.032575e+00
1116,Austria,Wheat,2019,kg,USD,Basic,"Compensation of employees; wages, salaries, & ...",emission/air,77ce2259-665c-43d9-bc70-215aaf1b09de,1.407558e+01
1117,Austria,Wheat,2019,kg,USD,Basic,"Compensation of employees; wages, salaries, & ...",emission/air,eed60168-63cc-4276-9b3d-c463528ea7c2,2.265663e+00
